**Importing libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mlt
%matplotlib inline
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df=pd.read_csv("../input/heart-attack-analysis-prediction-dataset/heart.csv")
df

In [ ]:
df.corr()

In [ ]:
df["age_group"] = pd.cut(x=df['age'], bins=[25,30,40,50,60,70,80], labels=["25-30", "30-40", '40-50', '50-60', '60-70', '70-80'])
df

**Reading data**

In [ ]:
df.hist(figsize=(20,10))
plt.show()

In [ ]:
df_ao = df.groupby(['age_group', 'output'])['output'].count()
df_ao

In [ ]:
l = []
for i in df_ao.index:
    if i[0] not in l:
        l.append(i[0])

In [ ]:
fig, ax = plt.subplots()
rects1 = ax.bar(l, df_ao[:, 0], label='0')
ax.set_title('age_group__output_0')
ax.legend();

In [ ]:
fig, ax = plt.subplots()
rects1 = ax.bar(l, df_ao[:, 1], label='1')
ax.set_title('age_group__output_1')
ax.legend();

In [ ]:
def counting_the_right(df, column_list):
    data = df.groupby(column_list).count()
    data = data.fillna(0)
    d = pd.DataFrame()
    for i in range(len(column_list)):
        d[column_list[i]] = np.array(list(data.index))[:, i]
    d['Count'] = data[data.columns[0]].values
    #d = d.fillna(0)
    return d

In [ ]:
column = ['age_group', 'sex', 'output']
d = counting_the_right(df, column)
d

In [ ]:
fig = px.sunburst(d, path=['age_group', 'sex', 'output'], values='Count')
fig.show()

In [ ]:
column = ['cp', 'output']
d = counting_the_right(df, column)

In [ ]:
fig = px.sunburst(d, path=['cp', 'output'], values='Count')
fig.show()

In [ ]:
d = {}
for i in list(df.columns):
    d[i] = df[i].value_counts().shape[0]
pd.DataFrame(d, index=['unique_counts']).T

In [ ]:
def circles(data, columns):
    for i in columns:
        c = data.loc[:, i].value_counts()
        fig = go.Figure()
        fig.add_trace(go.Pie(values=c, labels=c.index))
        fig.update_layout(title=i, title_x=0.5)
        fig.show()

In [ ]:
df_columns = ['cp', 'sex', 'fbs', 'restecg', 'exng', 'slp', 'caa', 'thall', 'output']

In [ ]:
circles(df, df_columns)

In [ ]:
df_1 = df.query("output == 1")

In [ ]:
circles(df_1, df_columns)

In [ ]:
df_columns2 = ['age', 'trtbps', 'chol', 'thalachh', 'oldpeak']

In [ ]:
df[df_columns2].describe().T

In [ ]:
pd.crosstab(df.age_group, df.output).plot(kind='bar');

In [ ]:
for i in df_columns:
    pd.crosstab(df[i], df.output).plot(kind='bar')
    plt.title(i + '_count')
    plt.xlabel(i)
    plt.ylabel('count')

**Prediction**

In [ ]:
X = df.drop(['output', 'age_group'], axis=1)
y = df.output

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.33)

In [ ]:
svc = make_pipeline(StandardScaler(), SVC(probability=True))

In [ ]:
svc.fit(x_train, y_train)

In [ ]:
data = pd.DataFrame()
def adequacy_check(df, check, model):
    pred = model.predict(x_test)
    pred_proba = model.predict_proba(x_test)[:, 1]
    df_results = pd.DataFrame(columns=['roc_auc'], data=[0])
    df_results['roc_auc'] = roc_auc_score(y_test, pred_proba)
    df_results['precision'] = precision_score(y_test, pred)
    df_results['f1'] = f1_score(y_test, pred)
    df_results['precision_0'] = precision_score(y_test, pred, pos_label=0)
    df_results['recall_0'] = recall_score(y_test, pred, pos_label=0)
    df_results['precision_1'] = precision_score(y_test, pred, pos_label=1)
    df_results['recall_1'] = recall_score(y_test, pred, pos_label=1)
    df_results.index = [check]
    df = df.append(df_results)
    return df

In [ ]:
data = adequacy_check(data, 'SVC', svc)
data

In [ ]:
tree = DecisionTreeClassifier(max_depth=3)
tree.fit(x_train, y_train)

In [ ]:
data = adequacy_check(data, 'tree', tree)
data

In [ ]:
random_tree = RandomForestClassifier()
random_tree.fit(x_train, y_train)

In [ ]:
data = adequacy_check(data, 'random_tree', random_tree)
data